In [1]:
###################################
# STAGE 1: Image Collection Composition
# Stage 1 creates the pre-processed input dataset (Image Collection)
# which will serve as the basis for or GeOBIA-base classification


# This code was adapted from learning materials provided by Iryna Dronova, PhD to 
# perform GeOBIA workflow to automate green space classification
# in the San Francisco Bay Area
# Author: Eric Romero -- PhD Student, ESPM, UC Berkeley
# Input data at time of last update:
# National Agricultural Imagery Program (NAIP)
# Senitnel-1 SAR (VH, VV)
# Sentinel-2 Multispectral
# Last update: 05/16/2023
######################################
    
# NOTE (Eric): Library imports and map visualization
import ipyleaflet
import ee
import geemap

ee.Initialize()


In [2]:
#NOTE (Eric): Import the assets will use for our AOI and class training
#NOTE (Eric): Asset names
BayAreaGroupsName = 'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/BayAreaBlockGroupsDissolved'
TrainName =  'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/BayAreaGreenSpaceTrainSmall'
ValName = 'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/BayAreaGreenSpaceValidateSmall'
BayAreaGridName = 'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/10kmGridSubset'

#TrainName =  'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/TestAOIval'
#ValName = 'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/TestAOItest'
#BayAreaGridName = 'projects/earthengine-legacy/assets/users/ericromero/eej/vectors/TestAOI'

# NOTE (Eric): Import assets as FeatureCollection objects 
BayAreaGroups = ee.FeatureCollection(BayAreaGroupsName) # NOTE (Eric): Bay Area AOI
trainpnts = ee.FeatureCollection(TrainName) # NOTE (Eric): Training data
valpnts = ee.FeatureCollection(ValName) # NOTE (Eric): Validation data
BayAreaGrid = ee.FeatureCollection(BayAreaGridName)


In [3]:
#NOTE (Eric): Pre-define functions we will use in image formation
# and classification

# NOTE (Eric): NAIP NDVI
def NAIPaddNDVI(image):
    
    ndvi = image.expression('(NIR - RED) / (NIR + RED)',{
        'RED': image.select('R'),
        'NIR': image.select('N')}).rename('NDVI')
    
    return(image.addBands(ndvi, overwrite=True))

# NOTE (Eric): NAIP EVI
def NAIPaddEVI(image):
    evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',{
                           'NIR': image.select('N'),
                           'RED': image.select('R'),
                           'BLUE': image.select('B')}).rename('EVI')
    
    return(image.addBands(evi))

# NOTE (Eric): Sentinel-2 NDVI
def S2addNDVI(image):
    
    ndvi = image.expression('(NIR - RED) / (NIR + RED)',{
        'RED': image.select('B4').divide(10000),
        'NIR': image.select('B8').divide(10000)}).rename('NDVI').copyProperties(image,['system:time_start'])
    
    return(image.addBands(ndvi,overwrite=True))

# NOTE (Eric): Sentinel-2 Bare Soil Index (BSI)
def S2addBSI(image):
    bsi = image.expression('((RED + SWIR) - (NIR + BLUE)) / ((RED + SWIR) + (NIR + BLUE))',{
        'RED': image.select('B4').divide(10000),
        'BLUE': image.select('B2').divide(10000),
        'NIR': image.select('B8').divide(10000),
        'SWIR': image.select('B11').divide(10000)}).rename('BSI').copyProperties(image,['system:time_start'])
    
    return(image.addBands(bsi,overwrite=True))

#NOTE (Eric): Sentinel-1 pseudo-Radar vegetation Index (pRVI)
def S1addRVI(image):
    
    rvi = image.expression('sqrt(VV/(VV + VH)) * ( (4*VH) / (VV/VH))' , {
        'VH' : image.select('VH'),
        'VV' : image.select('VV')}).rename('RVI').copyProperties(image,['system:time_start'])
    
    return(image.addBands(rvi,overwrite=True))

def maskS2clouds(image):
    
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return(image.updateMask(mask).divide(10000))

In [9]:
#NOTE (Eric): Define function that creates statistics images from NAIP, S1, and S2 collections
# using Feature Collection geometry

def ImFromFeatCollection(FeatCollection):
    
    #NOTE (Eric): Define time period of interest, inBands, outBands
    S_period_of_interest = ee.Filter.date('2014-01-01', '2023-01-01')
    NAIP_period_of_interest = ee.Filter.date('2020-01-01', '2023-01-01')

    NAIPinBands = ["R","G","B","N"]
    S2inBands = ["B2","B3","B4","B6","B8","B11"]
    S1inBands = ["VH", "VV"]

    ExtraOutBands = ["NAIP_NDVI_MEAN", "NAIP_NDVI_STD", "NAIP_EVI_MEAN", "S2_NDVI_MEAN",
                    "S2_NDVI_MAX", "S2_NDVI_STD", "S2_BSI_MEAN", "S2_BSI_MAX", "S2_BSI_STD"]#,
                    #"S1_RVI_MAX", "S1_RVI_MEAN"]
        
    outBands = []
    outBands.extend(NAIPinBands)
    outBands.extend(S2inBands)
    #outBands.extend(S1inBands)
    outBands.extend(ExtraOutBands)

    print(f'\nOutput dataset bands: {outBands}')
    print(f'\nNAIP inbands: {NAIPinBands}')
    
    # NOTE (Eric): Initialize ImageCollections
    S2_dataset = ee.ImageCollection('COPERNICUS/S2_SR').filter(S_period_of_interest).filterBounds(FeatCollection).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)).map(maskS2clouds)
    S1_dataset = ee.ImageCollection('COPERNICUS/S1_GRD').filter(S_period_of_interest).filterBounds(FeatCollection)
    NAIP_dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(NAIP_period_of_interest).filterBounds(FeatCollection)
    
    
    #NOTE (Eric): Filter Sentinel-1 collection for VV/VH polarizations and IW Swath mode
    allPolIW = S1_dataset.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW'))
    
    #NOTE (Eric): Add calculated indices as bands to Collections
    naip_collection = NAIP_dataset.select(NAIPinBands).map(NAIPaddNDVI).map(NAIPaddEVI);
    s2_collection = S2_dataset.select(S2inBands).map(S2addNDVI).map(S2addBSI)
    s1_collection = allPolIW.select(S1inBands).map(S1addRVI)
    
    #NOTE (Eric): Calculate means, medians, std devs, and maxes of bands/indices over archive of each collection
    # Add calculated stats as individual bands to output image
    
    #NOTE (Eric): NAIP calculations
    naip_median = naip_collection.select(NAIPinBands).median().clipToCollection(FeatCollection)
    naip_ndvimean = naip_collection.select('NDVI').reduce(ee.Reducer.mean()).rename("NAIP_NDVI_MEAN").clipToCollection(FeatCollection)
    naip_ndvistd = naip_collection.select('NDVI').reduce(ee.Reducer.stdDev()).float().rename("NAIP_NDVI_STD").clipToCollection(FeatCollection)
    naip_evimean = naip_collection.select('EVI').reduce(ee.Reducer.mean()).rename("NAIP_EVI_MEAN").clipToCollection(FeatCollection)

    #NOTE (Eric): Sentinel-2 Calculations
    s2_median = s2_collection.select(S2inBands).median();
    s2_ndvimean = s2_collection.select('NDVI').reduce(ee.Reducer.mean()).rename("S2_NDVI_MEAN").clipToCollection(FeatCollection)
    s2_ndvistd = s2_collection.select('NDVI').reduce(ee.Reducer.stdDev()).float().rename("S2_NDVI_STD").clipToCollection(FeatCollection)
    s2_ndvimax = s2_collection.select('NDVI').reduce(ee.Reducer.max()).rename("S2_NDVI_MAX").clipToCollection(FeatCollection)

    s2_bsimean = s2_collection.select('BSI').reduce(ee.Reducer.mean()).rename("S2_BSI_MEAN").clipToCollection(FeatCollection)
    s2_bsistd = s2_collection.select('BSI').reduce(ee.Reducer.stdDev()).float().rename("S2_BSI_STD").clipToCollection(FeatCollection)
    s2_bsimax = s2_collection.select('BSI').reduce(ee.Reducer.max()).rename("S2_BSI_MAX").clipToCollection(FeatCollection)

    #NOTE (Eric): Sentinel-1 Calculations
    s1_median = s1_collection.select(S1inBands).median();
    s1_rvimean = s1_collection.select('RVI').reduce(ee.Reducer.mean()).rename("S1_RVI_MEAN").clipToCollection(FeatCollection)
    s1_rvistd = s1_collection.select('RVI').reduce(ee.Reducer.stdDev()).float().rename("S1_RVI_STD").clipToCollection(FeatCollection)
    s1_rvimax = s1_collection.select('RVI').reduce(ee.Reducer.max()).rename("S1_RVI_MAX").clipToCollection(FeatCollection)
    
    #NOTE (Eric): Add the index statistic bands to the median bands from original imagery 
    # and clip the dataset with the ROI
    compclip = naip_median.addBands(naip_ndvimean).addBands(naip_ndvistd).addBands(naip_evimean).addBands(s2_median).addBands(s2_ndvimean).addBands(s2_ndvistd).addBands(s2_ndvimax).addBands(s2_bsimean).addBands(s2_bsistd).addBands(s2_bsimax).addBands(s1_median).addBands(s1_rvimean).addBands(s1_rvistd).addBands(s1_rvimax).clipToCollection(FeatCollection)
    
    #NOTE (Eric): Select and extract the images with the desidered bands defined as outBands
    final_bands = ee.Image(compclip.select(outBands)).toFloat(); #select only the bands specified earlier in the outBands
    
    return(final_bands)

In [7]:
###########################################
# STAGE 2: GeOBIA Classification
# This code was adapted from methods developed by Tassi A., Vizzari M., 2020,
# “Object-oriented LULC classification in Google Earth Engine combining SNIC, GLCM, and Machine Learning algorithms”,
# Remote Sens. 2020, 12(22), 3776; https://doi.org/10.3390/rs12223776.
# This code is intended for developing object-based image classification of urban green space 
# in the greater San Francisco Bay Area


# NOTE FROM THE AUTHORS (above doi) FOR THIS PART
# 1)Input requirements
# - roi: region of interest
# - trainpnts: feature collection containing all the training data
# - valpnts: validation points randomly generated and manually labelled used to assess the output
# accuracy
# - dataset: previously generated dataset using the following code:
# "https://code.earthengine.google.com/3b02d59f8dd400c450e380cc830247a2"

# 2) Pixel-based Approach
# Perfoms a pixel-based classification using the bands selected between those available in the
# dataset previosly generated
# bands= dataset.bandNames()

# 3) Object-oriented Approach
# Perfoms an object-oriented approach using the same band selected in the previous approach
# - The user can be set the variable "size_segmentation" that is the superpixel seed location spacing,
# in pixels.
# - Prediction bands used for the classification.
###################################################
# NOTE (Eric): Define a function that trains a Random Forest Classifier using SNIC GeOBIA, and
# the user-provided training/imagery
def TrainRF(image, trainpnts):
    
    
  
    # NOTE (Eric): Extract classification bands from input image
    classBands = image.bandNames()

    #NOTE (Eric): Select the bands from classBands var
    dataset = image.select(classBands)

    # ---- GeOBIA Training ----
    # NOTE (Iryna): Specify the spacing of the 'superpixel' seeds for initial object creation
    # by setting the superpixel seed location spacing, in pixels: (5 - 10 - 15 - 20)
    size_segmentation = 10

    # NOTE (Iryna): Segmentation using a SNIC approach based on the dataset previosly generated
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation) # to get spaced grid notes ata distance specified by segmentation size parameter
    snic = ee.Algorithms.Image.Segmentation.SNIC(image = dataset, # our multi-band image with selected bands same as for pixel-based
                                                compactness = 0, # allow flexibility in object shape, no need to force compactness
                                                connectivity = 8, # use all 8 neighboring pixels in a pixel neighborhood
                                                neighborhoodSize = 64,
                                                seeds = seeds)

    # NOTE (Iryna): The next step generates a list of band names from the snic image, but without "clusters"
    # since we don't need to use pixel values of their cluster IDs as a basis for class mapping:
    predictionBands = snic.bandNames().remove("clusters")

    # NOTE (Iryna): Classification using the classifier with the training bands called predictionBands
    # get training samples from the modified image with 'object'-mean pixel values:
    training_geobia = snic.select(predictionBands).sampleRegions(collection = trainpnts,
                                                                properties =  ['LULC'],
                                                                scale = 5)

    # NOTE (Iryna): Specify our random forest classifier and train:
    RF = ee.Classifier.smileRandomForest(50).train(features = training_geobia,
                                                  classProperty = 'LULC',
                                                  inputProperties = predictionBands)

    # NOTE (Eric): apply the trained model
    #classy_RF = snic.select(predictionBands).classify(RF,'Classified')
    #classyBandNames = classy_RF.bandNames()
    #class_band = classy_RF.select(['Classified']).toByte()

    return(RF)

#NOTE (Eric): Define a function that splits our image by grid
def gridSplit(i):
    
    filt = BayAreaGrid.filter(ee.Filter.eq('ID',i))
    grid_im = Im.clip(filt)
    return(grid_im)

#NOTE (Eric): Define a function that that classifies a feature collection using SNIC GeOBIA and pre-trained Random Forest
def ClassifyGeOBIA(image):
    # NOTE (Iryna): Specify the spacing of the 'superpixel' seeds for initial object creation
    # by setting the superpixel seed location spacing, in pixels: (5 - 10 - 15 - 20)
    size_segmentation = 10

    # NOTE (Iryna): Segmentation using a SNIC approach based on the dataset previosly generated
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation) # to get spaced grid notes ata distance specified by segmentation size parameter
    snic = ee.Algorithms.Image.Segmentation.SNIC(image = image, # our multi-band image with selected bands same as for pixel-based
                                                compactness = 0, # allow flexibility in object shape, no need to force compactness
                                                connectivity = 8, # use all 8 neighboring pixels in a pixel neighborhood
                                                neighborhoodSize = 64,
                                                seeds = seeds)
    #NOTE (Eric): Extract bands used for classification
    predictionBands=snic.bandNames().remove("clusters")

    #NOTE (Eric): Apply the trained Random Forest
    snic = snic.select(predictionBands).classify(BayAreaRF)
    snic = snic.toByte()
    classified = ee.Image(snic.select('classification')).toByte()
    
    return(classified)


In [10]:
# NOTE (Eric): Create image to use for training w/ composited stats
Im = ImFromFeatCollection(BayAreaGrid)

#NOTE (Eric): Train RF model
BayAreaRF = TrainRF(Im, trainpnts)

#NOTE (Eric): Create sequence object to form collection over
seq = ee.List.sequence(0,384)

#NOTE (Eric): Use sequence function to form image collection of original input dataset but clipped over
# series of smaller regions
ImCollection = ee.ImageCollection.fromImages(seq.map(gridSplit))

#NOTE (Eric): Iterate over the image collection, classify each image, 
# and write its classification to drive
ImCollectionList = ImCollection.toList(385)

out_fn_ext = '{}_EEJClassificationR2'

for i in range(385):
    
    #NOTE (Eric): Format export fn with feature ID
    export_fn = out_fn_ext.format(i)
    
    #NOTE (Eric): Get feature, geometry, and image from lists
    gridCollection = ee.FeatureCollection(BayAreaGrid.filter(ee.Filter.eq('ID',i)))
    gridList = ee.List(gridCollection.toList(1))
    grid = ee.Feature(gridList.get(0))
    grid_geom = ee.Geometry(grid.geometry())
    im = ee.Image(ImCollectionList.get(i))
    
    classy = ee.Image(ClassifyGeOBIA(im)).select('classification').toByte().clip(grid_geom)
    
    task = ee.batch.Export.image.toDrive(image=classy,
                                         description=export_fn,
                                         folder='EEJClassificationR2',
                                         region = grid_geom,
                                         scale=5,
                                         skipEmptyTiles=True,
                                         shardSize=64,
                                         maxPixels=1e13)
    task.start()

#NOTE (Eric): Input image visualization
#Map = geemap.Map(center=[37.7739,-122.4312],zoom=10)
#Map.addLayer(Im, {'bands': ['R','G','B'], 'min':[0,0,0], 'max': [255,255,255]})
#Map


Output dataset bands: ['R', 'G', 'B', 'N', 'B2', 'B3', 'B4', 'B6', 'B8', 'B11', 'NAIP_NDVI_MEAN', 'NAIP_NDVI_STD', 'NAIP_EVI_MEAN', 'S2_NDVI_MEAN', 'S2_NDVI_MAX', 'S2_NDVI_STD', 'S2_BSI_MEAN', 'S2_BSI_MAX', 'S2_BSI_STD']

NAIP inbands: ['R', 'G', 'B', 'N']
ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "ImageCollection.load",
                "arguments": {
                  "id": {
                    "constantValue": "USDA/NAIP/DOQQ"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.dateRangeContains",
                "arguments": {
                  "leftValue": {
 

In [8]:
#NOTE (Eric): RUN THIS AND REPLACE MISSING INDEX LIST WITH TILES WHOSE COMPUTATION TIMED OUT
# ONLY RUN IF MISSING TILES FROM ABOVE EXPORT
missing_indices = [128]

# NOTE (Eric): Create image to use for training w/ composited stats
Im = ImFromFeatCollection(BayAreaGrid)

#NOTE (Eric): Train RF model
BayAreaRF = TrainRF(Im, trainpnts)

#NOTE (Eric): Create sequence object to form collection over
seq = ee.List.sequence(0,384)

#NOTE (Eric): Use sequence function to form image collection of original input dataset but clipped over
# series of smaller regions
ImCollection = ee.ImageCollection.fromImages(seq.map(gridSplit))

#NOTE (Eric): Iterate over the image collection, classify each image, 
# and write its classification to drive
ImCollectionList = ImCollection.toList(385)

out_fn_ext = '{}_EEJClassificationR2'

for i in missing_indices:
    
    #NOTE (Eric): Format export fn with feature ID
    export_fn = out_fn_ext.format(i)
    
    #NOTE (Eric): Get feature, geometry, and image from lists
    gridCollection = ee.FeatureCollection(BayAreaGrid.filter(ee.Filter.eq('ID',i)))
    gridList = ee.List(gridCollection.toList(1))
    grid = ee.Feature(gridList.get(0))
    grid_geom = ee.Geometry(grid.geometry())
    im = ee.Image(ImCollectionList.get(i))
    
    classy = ee.Image(ClassifyGeOBIA(im)).select('classification').toByte().clip(grid_geom)
    
    task = ee.batch.Export.image.toDrive(image=classy,
                                         description=export_fn,
                                         folder='EEJClassificationR2',
                                         region = grid_geom,
                                         scale=5,
                                         skipEmptyTiles=True,
                                         shardSize=64,
                                         maxPixels=1e13)
    task.start()


Output dataset bands: ['R', 'G', 'B', 'N', 'B2', 'B3', 'B4', 'B6', 'B8', 'B11', 'NAIP_NDVI_MEAN', 'NAIP_NDVI_STD', 'NAIP_EVI_MEAN', 'S2_NDVI_MEAN', 'S2_NDVI_MAX', 'S2_NDVI_STD', 'S2_BSI_MEAN', 'S2_BSI_MAX', 'S2_BSI_STD']

NAIP inbands: ['R', 'G', 'B', 'N']
